# VacationPy
----

#### Note
* Keep an eye on your API usage. Use https://developers.google.com/maps/reporting/gmp-reporting as reference for how to monitor your usage and billing.

* Instructions have been included for each segment. You do not have to follow them exactly, but they are included to help you think through the steps.

In [1]:
# Dependencies and Setup
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
import requests
import gmaps
import os
import json

# Import API key
from api_keys import g_key

gmaps.configure(api_key=g_key)

### Store Part I results into DataFrame
* Load the csv exported in Part I to a DataFrame

In [2]:
vacation_df = pd.read_csv('../output_data/cities.csv')
vacation_df.head()

,Unnamed: 0,City,Lat,Lng,High Temp,Humidity,Cloudiness,Wind Speed,Country,Date Time
0,0,new norfolk,-42.78,147.06,54.00,58,23,1.01,au,10-13-2020
1,1,norrtalje,59.76,18.70,46.99,85,43,4.00,se,10-13-2020
2,2,mayo,38.89,-76.51,63.00,93,75,13.47,ca,10-13-2020
3,3,sola,-13.88,167.55,81.81,70,59,1.92,vu,10-13-2020
4,5,dzhebariki-khaya,62.22,135.80,25.23,95,99,8.75,ru,10-13-2020


### Humidity Heatmap
* Configure gmaps.
* Use the Lat and Lng as locations and Humidity as the weight.
* Add Heatmap layer to map.

In [3]:
# Store latitude and longitude in locations
locations = vacation_df[['Lat','Lng']]

# Filla NaN values and convert to float
humidity = vacation_df['Humidity'].astype(float)
locations.head()

,Lat,Lng
0,-42.78,147.06
1,59.76,18.70
2,38.89,-76.51
3,-13.88,167.55
4,62.22,135.80


In [4]:
# Plot Heatmap
fig = gmaps.figure()

# create heat layer
humidity_layer = gmaps.heatmap_layer(locations, weights = humidity,
                                dissipating = False, max_intensity = 10,
                                point_radius = 1)
# add the layer to the map
fig.add_layer(humidity_layer)

fig

Figure(layout=FigureLayout(height='420px'))

### Create new DataFrame fitting weather criteria
* Narrow down the cities to fit weather conditions.
* Drop any rows will null values.

In [5]:
hotel_df = vacation_df.loc[(vacation_df['High Temp'] < 75) & (vacation_df['High Temp'] > 45)\
                             & (vacation_df['Lat'] < 75) & (vacation_df['Cloudiness'] < 40) & \
                             (vacation_df['Humidity'] < 60)]
hotel_df.head()

,Unnamed: 0,City,Lat,Lng,High Temp,Humidity,Cloudiness,Wind Speed,Country,Date Time
0,0,new norfolk,-42.78,147.06,54.00,58,23,1.01,au,10-13-2020
65,70,new norfolk,-42.78,147.06,54.00,58,23,1.01,au,10-13-2020
68,74,san luis,-33.30,-66.34,72.28,25,0,5.75,ar,10-13-2020
76,82,jamestown,42.10,-79.24,55.99,58,1,10.29,sh,10-13-2020
80,86,new norfolk,-42.78,147.06,54.00,58,23,1.01,au,10-13-2020


### Hotel Map
* Store into variable named `hotel_df`.
* Add a "Hotel Name" column to the DataFrame.
* Set parameters to search for hotels with 5000 meters.
* Hit the Google Places API for each city's coordinates.
* Store the first Hotel result into the DataFrame.
* Plot markers on top of the heatmap.

In [6]:
# base url to pull maps data
base_url = "https://maps.googleapis.com/maps/api/place/nearbysearch/json"
hotel_df['Hotel Name'] = None
hotel_df['Rating'] = 0

# set parameters for search
# for loop to find each location from the df

for row in hotel_df.index:

    # set the lat/long correctly
    lat = (hotel_df['Lat'][row])
    long = (hotel_df['Lng'][row])
    lat_long = f"{lat},{long}"
                            #print(type(lat_long)) --> use for testing

    # create the location parameter and set lat_long as the coords
    parameters = {                       
         "location": lat_long,
         "radius": 5000,
         "type": "lodging",
         "key": g_key
         } 

    # request the hotel at the coords
    response = requests.get(base_url, params=parameters)
    answer = response.json()
    
    try:
        hotel_df.loc[row, 'Hotel Name'] = answer['results'][0]['name']
        hotel_df.loc[row, 'Rating'] = answer['results'][0]['rating']  # i tried to get the hotel rating to add to the info
 
    except IndexError:
        hotel_df.loc[row, 'Hotel Name'] = 'No hotel'
        hotel_df.loc[row, 'Rating'] = 0
    except KeyError:
        hotel_df.loc[row, 'Hotel Name'] = 'No hotel'
        hotel_df.loc[row, 'Rating'] = 0       
        
# print(json.dumps(answer, indent=4, sort_keys=True)) #--> use this to see the json layout

hotel_df

# note that i get a red box with a warning.  Pandas wants to use .loc in order to input the data into the df.
# i checked the documentation on this and changed my code to use .loc.  But I'm still getting this red box
# with the warning.  My df is populating correctly, so I'm not sure what I'm doing wrong.

<ipython-input-6-0f8f003b974f>:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  hotel_df['Hotel Name'] = None
<ipython-input-6-0f8f003b974f>:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  hotel_df['Rating'] = 0
C:\Users\cro11\anaconda3\lib\site-packages\pandas\core\indexing.py:966: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/st

{
    "html_attributions": [],
    "results": [],
    "status": "ZERO_RESULTS"
}


,Unnamed: 0,City,Lat,Lng,High Temp,Humidity,Cloudiness,Wind Speed,Country,Date Time,Hotel Name,Rating
0,0,new norfolk,-42.78,147.06,54.00,58,23,1.01,au,10-13-2020,The Shingles Riverside Cottages,4.7
65,70,new norfolk,-42.78,147.06,54.00,58,23,1.01,au,10-13-2020,The Shingles Riverside Cottages,4.7
68,74,san luis,-33.30,-66.34,72.28,25,0,5.75,ar,10-13-2020,Huarpes Hotel,3.8
76,82,jamestown,42.10,-79.24,55.99,58,1,10.29,sh,10-13-2020,Hampton Inn & Suites Jamestown,4.4
80,86,new norfolk,-42.78,147.06,54.00,58,23,1.01,au,10-13-2020,The Shingles Riverside Cottages,4.7
82,88,new norfolk,-42.78,147.06,54.00,58,23,1.01,au,10-13-2020,The Shingles Riverside Cottages,4.7
109,118,jamestown,42.10,-79.24,55.99,58,1,10.29,sh,10-13-2020,Hampton Inn & Suites Jamestown,4.4
130,140,gobabis,-22.45,18.97,65.73,27,0,12.46,na,10-13-2020,Goba Lodge,4.2
156,169,borjomi,41.85,43.41,57.83,39,6,4.99,ge,10-13-2020,Likani Palace,3.9
157,170,new norfolk,-42.78,147.06,54.00,58,23,1.01,au,10-13-2020,The Shingles Riverside Cottages,4.7


In [7]:
# hotel_list = []
# ratings = []

# if a hotel is in the vicinity, append; if not, append "No Hotel" and no rating
#     try:
#         hotel_list.append(answer['results'][0]['name'])
#         ratings.append(answer['results'][0]['rating'])
#     except IndexError:
#         hotel_list.append('No hotel')
#         ratings.append(0)


# Add the lists to the df
# hotel_df['Hotel Name'] = hotel_list
# hotel_df['Rating'] = ratings

In [8]:
# # NOTE: Do not change any of the code in this cell

# # Using the template add the hotel marks to the heatmap
# info_box_template = """
# <dl>
# <dt>Name</dt><dd>{Hotel Name}</dd>
# <dt>City</dt><dd>{City}</dd>
# <dt>Country</dt><dd>{Country}</dd>
# </dl>
# """
# # Store the DataFrame Row
# # NOTE: be sure to update with your DataFrame name
# hotel_info = [info_box_template.format(**row) for index, row in hotel_df.iterrows()]
# locations = hotel_df[["Lat", "Lng"]]

In [10]:
# I didn't use .iterrows in my solution, so the above code won't work for me.  I 
# 
# Add marker layer ontop of heat map

hotel_coords = hotel_df[['Lat', 'Lng']]

info_box_template = """
<dl>
<dt>Hotel Name:</dt><dd>{Hotel Name}</dd>
<dt>City: </dt><dd>{City}</dd>
<dt>Country Code: </dt><dd>{Country}</dd>
<dt>Rating: </dt><dd>{Rating}</dd>
</dl>
"""
hotel_info = [info_box_template.format(**row) for index, row in hotel_df.iterrows()]

markers = gmaps.marker_layer(hotel_coords, info_box_content = hotel_info)

# Add the layer to the map
fig.add_layer(markers)
fig

# Display figure


Figure(layout=FigureLayout(height='420px'))